<a href="https://colab.research.google.com/github/theQuert/inlpfun/blob/master/Demo/summary_parser_MDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers
import re
import urllib
from urllib import request
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [3]:
#@title Keywords input
keywords = 'pfizer children' #@param {type: "string"}
keywords = re.sub(' ', '+', keywords)
start_date = '2021-11-03' #@param {type: "string"}
end_date = '2021-11-05' #@param {type: "string"}
search_inputs = f'{keywords}+after%3A{start_date}+before%3A{end_date}'
url = f"https://www.google.com/search?q={search_inputs}&tbm=nws&lr=lang_en&hl=en&sort=date&num=19"
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
}

In [4]:
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.content, "html.parser")
soup_prettify = soup.prettify()
with open('url.html', 'w') as fwrite:
    fwrite.write(soup_prettify)

In [5]:
url_list = []
metas = []
for link in soup.find_all('a'):
    url_list.append(link.get('href'))
for url in url_list:
    if bool(re.findall('google.com', str(url)))==True:
        url_list.remove(url)
regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
for url in url_list:
    if bool(re.findall(regex, str(url)))==True:
        metas.append(url)
for meta in metas:
    if bool(re.findall('policies.google.com', str(meta)))==True:
        metas.remove(meta)

In [6]:
metas

['https://www.doh.wa.gov/Newsroom/Articles/ID/2957/Children-ages-5-11-now-eligible-for-Pfizer-BioNTech-COVID-19-vaccine',
 'https://www.jdnews.com/story/news/2021/11/04/pfizer-covid-vaccine-available-kids-onslow-county/6264687001/',
 'https://www.timesherald.com/2021/11/04/montgomery-county-scheduling-pfizer-covid-19-vaccinations-for-children-ages-5-11',
 'https://www.nj.gov/health/news/2021/approved/20211103a.shtml',
 'https://www.tn.gov/health/news/2021/11/3/pfizer-covid-19-vaccine-approved-for-ages-5-11.html',
 'https://shawneemissionpost.com/2021/11/04/johnson-county-chidlren-vaccine-135979/',
 'https://www.mecknc.gov/news/Pages/Pfizer%20COVID-19%20Pediatric%20Vaccine%20for%20Children%20Ages%205-11%20Now%20Available.aspx',
 'https://www.kshb.com/news/coronavirus/kansas-city-area-locations-offering-the-pfizer-vaccine-for-children-ages-5-11',
 'https://www.npr.org/2021/11/04/1052366451/parents-covid-vaccine-kids-5-11',
 'https://www.euronews.com/next/2021/11/03/pfizer-s-covid-vaccine

In [ ]:
inputs = []
for meta in metas:
    resp = requests.get(meta, headers=headers, verify=False)
    soup = BeautifulSoup(resp.content, "html.parser")
    title = soup.title.string
    title_filtered = re.sub(r'(\s)-(\s)(\w+).+', '', title)
    title_filtered = title_filtered.strip('\r\n\t')
    inputs.append(title_filtered)

In [8]:
summarizer = pipeline("summarization")
summ=' '.join([str(i) for i in inputs])
summarized = summarizer(summ, min_length=5, max_length=20)
print(summarized)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

[{'summary_text': " Pfizer's COVID vaccine can now be used on children aged 5-11 in"}]
